In [3]:
import numpy as np
import pandas as pd
import time

import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

import pickle5 as pickle

import matplotlib.ticker as tick

import sys
sys.path.append('..')

from nsgp_vi import nsgpVI

# We'll use double precision throughout for better numerics.
dtype = np.float64

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

plt.style.use('ggplot') 
plt.style.use('seaborn-paper') 
plt.style.use('seaborn-whitegrid') 

In [ ]:
#plot the synthetic data for 1 individual, including lengthscale and the amplitude 

df = pd.read_csv('../data/ns_synthetic_data_indv_1_0.csv')
T = df['Time'].values[:,None]
X = np.array(df['observations']).reshape(len(T),1)

true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

plt.plot(T,true_len,'.',markersize=1)
plt.xlabel('Time')
plt.ylabel('Lengthscale')
plt.show()

plt.plot(T,true_var,'.',markersize=1)
plt.xlabel('Time')
plt.ylabel('Variance')
plt.show()

# Multiple individuals OU data with added noise
plt.plot(T,X,'.',markersize=1)
plt.xlabel('Time')
plt.ylabel('Observations')
plt.show()

In [4]:
        df = pd.read_csv('../data/ns_synthetic_data_indv_128_0.csv')
        T = df['Time'].values[:,None]
        ID = df['ID'].values
        X = np.array(df['observations']).reshape(len(T),1)
        true_len = np.array(df['Lengthscale']).reshape(len(T),1)
        true_var = np.array(df['Variance']).reshape(len(T),1)
        
        num_training_points_ = T.shape[0]
        num_inducing_points_ = 50
        inducing_index_points = np.linspace(0., 60*24., num_inducing_points_, endpoint=False)[..., np.newaxis]
        np.random.shuffle(inducing_index_points)
        
        BATCH_SIZE=8
        SEG_LENGTH=1024
        allT = []
        allX = []
        
        for i in np.unique(df['ID'].values):
            allT.append(df['Time'][df['ID'].values == i].values[...,None])
            allX.append(df['observations'][df['ID'].values == i].values[...,None])

        class segment_generator:
            def __iter__(self):
                # loop over individuals
                self.i = 0
                self.max_i = len(allT)
                
                # loop over segments
                self.j = 0
                self.max_j = num_training_points_//(self.max_i*SEG_LENGTH)
            
                return self
            
            def __next__(self):
                
                if self.i == self.max_i:
                    self.i = 0
                    self.j +=1
                    if self.j==self.max_j:
                        raise StopIteration
                
                T = allT[self.i]
                X = allX[self.i] 

                TT = T[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
                XX = X[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
        
                self.i += 1

                return TT,XX
            
        dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
        dataset = dataset.map(lambda dd: (dd[0],dd[1]))
        dataset = dataset.shuffle(1000)
        dataset = dataset.batch(BATCH_SIZE)
        
        kernel_len_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
        kernel_len_l = tfp.util.TransformedVariable(30.0,tfb.Chain([tfb.Scale(np.float64(60.)),tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)
        
        # amplitude kernel parameters, lower levels
        kernel_amp_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(), dtype=tf.float64, name='k_amp_a',trainable=True)
        kernel_amp_l = tfp.util.TransformedVariable(30.0,tfb.Chain([tfb.Scale(np.float64(60.)),tfb.Softplus()]), dtype=tf.float64, name='k_amp_l',trainable=True)
        
        #kernels on the second layer
        kernel_len = tfk.ExponentiatedQuadratic(kernel_len_a,kernel_len_l)
        kernel_amp = tfk.ExponentiatedQuadratic(kernel_amp_a,kernel_amp_l)
        
        #print(str(nrd))

        vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  
        
         

In [5]:
vgp.trainable_variables

In [7]:
#plot all the plots at once
ZZ = np.linspace(0,24*60,200)[:,None]

fig, ax = plt.subplots(3, 2,figsize=(30,33))

ax00 = ax[0,0]
ax01= ax[0,1]

ax10 = ax[1,0]
ax11= ax[1,1]

ax20 = ax[2,0]
ax21= ax[2,1]

#n1

#load the data for n1, choose one of five

df = pd.read_csv('../data/ns_synthetic_data_indv_1_0.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../results/T_ind_n1_0.npy')

#load the model
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=None,num_training_points=num_training_points_, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n1_0.pkl', 'rb') as f:
    loadp = pickle.load(f)

for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

#plot for n1    
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5
    

ax00.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax00.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax00.set_xlabel('Time',size=30)
ax00.set_ylabel('Lengthscale',size=30)
ax00.text(-0.05,1,'A', size=40, transform=ax[0, 0].transAxes)
ax00.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)


ax01.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax01.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax01.set_xlabel('Time',size=30)
ax01.set_ylabel('Amplitude',size=30)
ax01.text(-0.05,1,'B', size=40, transform=ax[0, 1].transAxes)
ax01.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

#n8

#load the data

df = pd.read_csv('../data/ns_synthetic_data_indv_8_2.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

num_training_points = T.shape[0]


#load the inducing points
inducing_index_points = np.load('../results/T_ind_n8_2.npy')

#load the model
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=None,num_training_points=num_training_points, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n8_2.pkl', 'rb') as f:
    loadp = pickle.load(f)

for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

#plot for n8
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax10.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax10.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax10.set_xlabel('Time',size=30)
ax10.set_ylabel('Lengthscale',size=30)
ax10.text(-0.07,1,'C', size=40, transform=ax[1, 0].transAxes)
ax10.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

ax11.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax11.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax11.set_xlabel('Time',size=30)
ax11.set_ylabel('Amplitude',size=30)
ax11.text(-0.05,1,'D', size=40, transform=ax[1, 1].transAxes)
ax11.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

#n128 data

#load the data

df = pd.read_csv('../data/ns_synthetic_data_indv_128_3.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
num_training_points = T.shape[0]

#load the inducing points
inducing_index_points = np.load('../results/T_ind_n128_3.npy')   

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=None,num_training_points=num_training_points, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n128_3.pkl', 'rb') as f:
    loadp = pickle.load(f)    
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)   

#plot for n128
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])
    
len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5    
    
ax20.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax20.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax20.set_xlabel('Time',size=30)
ax20.set_ylabel('Lengthscale',size=30)
ax20.text(-0.07,1,'E', size=40, transform=ax[2, 0].transAxes)
ax20.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

ax21.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax21.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax21.set_xlabel('Time',size=30)
ax21.set_ylabel('Amplitude',size=30)
ax21.text(-0.05,1,'F', size=40, transform=ax[2, 1].transAxes)
ax21.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

fig.tight_layout()

plt.savefig("Sythetic_inference_vi.pdf",dpi=600)

plt.show()
